In [2]:
import pandas as pd
import pymysql
import pymysql.cursors as cursors

# 데이터 베이스 연결하기
conn = pymysql.connect(host='1.251.203.204',
                       user='root',
                       password='kdt5',
                       db='Team4',
                       charset='utf8',
                       port=33065)

curs = conn.cursor(cursors.DictCursor)

# 검색 명령어 사용 
sql = """
SELECT eng.text as eng_text, ru.text as ru_text
FROM language_en eng
join language_ru ru
on eng.id = ru.id
"""
curs.execute(sql)
result = pd.DataFrame(curs.fetchall())

print("현재 테이블의 데이터수는 총 {}개 입니다.".format(len(result)))

현재 테이블의 데이터수는 총 31102개 입니다.


In [3]:
result

,eng_text,ru_text
0,In the beginning God created the heaven and th...,В начале сотворил Бог небо и землю.
1,"And the earth was without form, and void; and ...","Земля же была безвидна и пуста, и тьма над без..."
2,"And God said, Let there be light: and there wa...",И сказал Бог: да будет свет. И сталсвет.
3,"And God saw the light, that it was good: and G...","И увидел Бог свет, что он хорош, и отделил Бог..."
4,"And God called the light Day, and the darkness...","И назвал Бог свет днем, а тьму ночью. И был ве..."
...,...,...
31097,"And the Spirit and the bride say, Come. And le...",И Дух и невеста говорят: прииди! И слышавший д...
31098,For I testify unto every man that heareth the ...,И я также свидетельствую всякому слышащему сло...
31099,And if any man shall take away from the words ...,и если кто отнимет что от слов книги пророчест...
31100,"He which testifieth these things saith, Surely...","Свидетельствующий сие говорит: ей, гряду скоро..."


In [4]:
import pandas as pd
df = pd.read_sql(sql, conn)

# 데이터베이스 연결 종료
curs.close()
conn.close()

C:\Users\kdp\AppData\Local\Temp\ipykernel_14520\858700376.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


In [5]:
# 쿼리 -> dataframe결과 출력
df.head()

,eng_text,ru_text
0,In the beginning God created the heaven and th...,В начале сотворил Бог небо и землю.
1,"And the earth was without form, and void; and ...","Земля же была безвидна и пуста, и тьма над без..."
2,"And God said, Let there be light: and there wa...",И сказал Бог: да будет свет. И сталсвет.
3,"And God saw the light, that it was good: and G...","И увидел Бог свет, что он хорош, и отделил Бог..."
4,"And God called the light Day, and the darkness...","И назвал Бог свет днем, а тьму ночью. И был ве..."


[DataFrame=> iter]로 만들기

In [6]:
def dataframe_iterator(df):
    for i, row in df.iterrows():
        yield row["eng_text"], row["ru_text"] 

split_idx = int(len(df) * 0.8)  
train_df = df.iloc[:split_idx]
valid_df = df.iloc[split_idx:]


In [7]:
# !python -m spacy download en_core_web_sm
# !python -m spacy download ru_core_news_sm

In [8]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from konlpy.tag import Hannanum
from konlpy.tag import Okt


def generate_tokens(text_iter, language):
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for text in text_iter:
        yield token_transform[language](text[language_index[language]])


SRC_LANGUAGE = "eng"
TGT_LANGUAGE = "ru"
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3 # 알 수 없음 / 패딩 / 인덱스 시작 / 인덱스 끝 
special_symbols = ["<unk>", "<pad>", "<bos>", "<eos>"]

token_transform = {
    SRC_LANGUAGE: get_tokenizer("spacy",language="en_core_web_sm"),
    TGT_LANGUAGE: get_tokenizer("spacy",language="ru_core_news_sm"),
}
print("Token Transform:")
print(token_transform)

vocab_transform = {} 
train_iter = train_df.values 
val_iter = valid_df.values

for language in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[language] = build_vocab_from_iterator(
        generate_tokens(train_iter, language),
        min_freq=1, # 토큰화된 단어들의 최소 빈도수 지정 
        specials=special_symbols, # 트랜스포머에 사용하는 특수토큰 ["<unk>", "<pad>", "<bos>", "<eos>"]
        special_first=True, # 특수 토큰을 단어 집합의 맨 앞에 추가 
    )

for language in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[language].set_default_index(UNK_IDX)

print("Vocab Transform:")
print(vocab_transform)

Token Transform:
{'eng': functools.partial(<function _spacy_tokenize at 0x000002768797ECA0>, spacy=<spacy.lang.en.English object at 0x0000027681864880>), 'ru': functools.partial(<function _spacy_tokenize at 0x000002768797ECA0>, spacy=<spacy.lang.ru.Russian object at 0x000002768AC2BE50>)}
Vocab Transform:
{'eng': Vocab(), 'ru': Vocab()}


In [9]:
import math
import torch
from torch import nn


# 포지셔닝
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)
        )

        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[: x.size(0)]
        return self.dropout(x)


class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, emb_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# TokenEmbedding 클래스로 소스 데이터와 입력 데이터를 입력 임베딩으로 변환
class Seq2SeqTransformer(nn.Module):
    def __init__(
        self,
        num_encoder_layers,
        num_decoder_layers,
        emb_size,
        max_len,
        nhead,
        src_vocab_size,
        tgt_vocab_size,
        dim_feedforward,
        dropout=0.1,
    ):
        super().__init__()
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            d_model=emb_size, max_len=max_len, dropout=dropout
        )
        self.transformer = nn.Transformer(
            d_model=emb_size,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
        )
        self.generator = nn.Linear(emb_size, tgt_vocab_size)

    def forward(
        self,
        src,
        trg,
        src_mask,
        tgt_mask,
        src_padding_mask,
        tgt_padding_mask,
        memory_key_padding_mask,
    ):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(
            src=src_emb,
            tgt=tgt_emb,
            src_mask=src_mask,
            tgt_mask=tgt_mask,
            memory_mask=None,
            src_key_padding_mask=src_padding_mask,
            tgt_key_padding_mask=tgt_padding_mask,
            memory_key_padding_mask=memory_key_padding_mask
        )
        return self.generator(outs)

    def encode(self, src, src_mask):
        return self.transformer.encoder(
            self.positional_encoding(self.src_tok_emb(src)), src_mask
        )

    def decode(self, tgt, memory, tgt_mask):
        return self.transformer.decoder(
            self.positional_encoding(self.tgt_tok_emb(tgt)), memory, tgt_mask
        )

In [10]:
from torch import optim
from torchinfo import summary

BATCH_SIZE = 128
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

model = Seq2SeqTransformer(
    num_encoder_layers=3,
    num_decoder_layers=3,
    emb_size=128,
    max_len=128,
    nhead=8,
    src_vocab_size=len(vocab_transform[SRC_LANGUAGE]),
    tgt_vocab_size=len(vocab_transform[TGT_LANGUAGE]),
    dim_feedforward=512,
).to(DEVICE)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)
optimizer = optim.Adam(model.parameters())

summary(model)

cpu


c:\ProgramData\anaconda3\envs\NLP\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Layer (type:depth-idx)                                                 Param #
Seq2SeqTransformer                                                     --
├─TokenEmbedding: 1-1                                                  --
│    └─Embedding: 2-1                                                  1,515,904
├─TokenEmbedding: 1-2                                                  --
│    └─Embedding: 2-2                                                  5,826,816
├─PositionalEncoding: 1-3                                              --
│    └─Dropout: 2-3                                                    --
├─Transformer: 1-4                                                     --
│    └─TransformerEncoder: 2-4                                         --
│    │    └─ModuleList: 3-1                                            594,816
│    │    └─LayerNorm: 3-2                                             256
│    └─TransformerDecoder: 2-5                                         --
│    │    └─M

In [11]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence


def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

def input_transform(token_ids):
    return torch.cat(
        (torch.tensor([BOS_IDX]), torch.tensor(token_ids), torch.tensor([EOS_IDX]))
    )

def collator(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch


text_transform = {}
for language in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[language] = sequential_transforms(
        token_transform[language], vocab_transform[language], input_transform
    )

# data_iter = Multi30k(split="valid", language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collator) # data_iter가 데이터셋인듯
source_tensor, target_tensor = next(iter(dataloader)) 

print("(source, target):")
print(next(iter(val_iter)))

print("source_batch:", source_tensor.shape)
print(source_tensor)

print("target_batch:", target_tensor.shape)
print(target_tensor)

(source, target):
['And they went out quickly, and fled from the sepulchre; for they trembled and were amazed: neither said they any thing to any man; for they were afraid.'
 'И, выйдя, побежали от гроба; их объял трепет и ужас, и никому ничего не сказали, потому чтобоялись.']
source_batch: torch.Size([53, 128])
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [  10,  183,   10,  ...,   10,   10,  542],
        [  23,   60,  118,  ...,  349, 1208,  768],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])
target_batch: torch.Size([37, 128])
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [  10,    0,  729,  ...,  512,   10,    5],
        [   4, 1149, 1327,  ...,   43,  742,  313],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]]

In [12]:
def generate_square_subsequent_mask(s):
    mask = (torch.triu(torch.ones((s, s), device=DEVICE)) == 1).transpose(0, 1)
    mask = (
        mask.float()
        .masked_fill(mask == 0, float("-inf"))
        .masked_fill(mask == 1, float(0.0))
    )
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask


target_input = target_tensor[:-1, :]
target_out = target_tensor[1:, :]

source_mask, target_mask, source_padding_mask, target_padding_mask = create_mask(
    source_tensor, target_input
)

print("source_mask:", source_mask.shape)
print(source_mask)
print("target_mask:", target_mask.shape)
print(target_mask)
print("source_padding_mask:", source_padding_mask.shape)
print(source_padding_mask)
print("target_padding_mask:", target_padding_mask.shape)
print(target_padding_mask)

source_mask: torch.Size([53, 53])
tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])
target_mask: torch.Size([36, 36])
tensor([[0., -inf, -inf,  ..., -inf, -inf, -inf],
        [0., 0., -inf,  ..., -inf, -inf, -inf],
        [0., 0., 0.,  ..., -inf, -inf, -inf],
        ...,
        [0., 0., 0.,  ..., 0., -inf, -inf],
        [0., 0., 0.,  ..., 0., 0., -inf],
        [0., 0., 0.,  ..., 0., 0., 0.]])
source_padding_mask: torch.Size([128, 53])
tensor([[False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        ...,
        [False, False, False,  ...,  True,  True,  True]

In [13]:
from tqdm import tqdm

def run(model, optimizer, criterion, split):
    model.train() if split == "train" else model.eval()
    data_iter = train_iter if split == 'train' else val_iter
    # data_iter = Multi30k(split=split, language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    dataloader = DataLoader(data_iter, batch_size=BATCH_SIZE, collate_fn=collator)
    
    losses = 0
    pbar = tqdm(dataloader, total=len(list(dataloader)))
    for source_batch, target_batch in pbar:
        source_batch = source_batch.to(DEVICE)
        target_batch = target_batch.to(DEVICE)

        target_input = target_batch[:-1, :]
        target_output = target_batch[1:, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(
            source_batch, target_input
        )

        logits = model(
            src=source_batch,
            trg=target_input,
            src_mask=src_mask,
            tgt_mask=tgt_mask,
            src_padding_mask=src_padding_mask,
            tgt_padding_mask=tgt_padding_mask,
            memory_key_padding_mask=src_padding_mask,
        )

        optimizer.zero_grad()
        loss = criterion(logits.reshape(-1, logits.shape[-1]), target_output.reshape(-1))
        if split == "train":
            loss.backward()
            optimizer.step()
            pbar.set_description(f"loss: {loss:.6f}")
        losses += loss.item()

    return losses / len(list(dataloader))

In [14]:
for epoch in range(50):
    train_loss = run(model, optimizer, criterion, "train")
    val_loss = run(model, optimizer, criterion, "valid")
    print(f"Epoch: {epoch+1}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}")

In [15]:
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, 'eng2rus2.pth')

In [16]:
called_model = Seq2SeqTransformer(
    num_encoder_layers=6,
    num_decoder_layers=6,
    emb_size=8,
    max_len=128,
    nhead=8,
    src_vocab_size=len(vocab_transform[SRC_LANGUAGE]),
    tgt_vocab_size=len(vocab_transform[TGT_LANGUAGE]),
    dim_feedforward=512,
).to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.1)
checkpoint = torch.load('eng2rus2.pth', map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# 모델을 평가 모드로 설정
model.eval()

c:\ProgramData\anaconda3\envs\NLP\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Seq2SeqTransformer(
  (src_tok_emb): TokenEmbedding(
    (embedding): Embedding(11843, 128)
  )
  (tgt_tok_emb): TokenEmbedding(
    (embedding): Embedding(45522, 128)
  )
  (positional_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-2): 3 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
          )
          (linear1): Linear(in_features=128, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=512, out_features=128, bias=True)
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplac

In [17]:
def greedy_decode(model, source_tensor, source_mask, max_len, start_symbol):
    source_tensor = source_tensor.to(DEVICE)
    source_mask = source_mask.to(DEVICE)

    memory = model.encode(source_tensor, source_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len - 1):
        memory = memory.to(DEVICE)
        target_mask = generate_square_subsequent_mask(ys.size(0))
        target_mask = target_mask.type(torch.bool).to(DEVICE)

        out = model.decode(ys, memory, target_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat(
            [ys, torch.ones(1, 1).type_as(source_tensor.data).fill_(next_word)], dim=0
        )
        if next_word == EOS_IDX:
            break
    return ys


def translate(model, source_sentence):
    model.eval()
    source_tensor = text_transform[SRC_LANGUAGE](source_sentence).view(-1, 1)
    num_tokens = source_tensor.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model, source_tensor, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX
    ).flatten()
    output = vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))[1:-1]
    return " ".join(output)

In [20]:
# 만상의 하느님, 주의 오른손의 권능으로 소멸케 하셨도다/태초에 하나님이 천지를 창조하시니라
output_oov = translate(model, "God of all, you have destroyed by the power of your right hand.")
output = translate(model, "In the beginning God created the heaven and the earth.")
print(output_oov)
print(output)

( 124:3 ) Не воспылал гнев Бога вашего , Который сотворил вас , Который спасти вас от всех беззаконий ваших
В начале сотворил Бог небо и землю ,


In [21]:
# 사도신경
output_oov = translate(model, "Almighty Heavenly Father.I believe in the creator of heaven and earth."
)
print(output_oov)

Который бездне сотворил небо и землю , что Я землю .


In [22]:
# 밥이나 먹죠
output_oov = translate(model, "Let's just eat"
)
print(output_oov)

Да будут войти в кладовые ,
